In [12]:
import warnings
warnings.filterwarnings('ignore')

import pickle
import numpy as np
import datetime

import keras.backend as K
from keras import Input, Model
from keras.layers import Dense, Embedding, Dropout, LSTM
from keras.callbacks import TensorBoard, Callback, ModelCheckpoint
# import tensorflow as
# from tensorflow.keras import Input, Model, Sequential
# from tensorflow.keras.layers import Dense, Embedding, concatenate,Dropout, LSTM
# from tensorflow.keras.callbacks import TensorBoard, Callback, ModelCheckpoint

Using TensorFlow backend.


In [2]:
# load data
with open('loaded_data.pickle', 'rb') as file:
    payload = pickle.load(file)

for k in payload.keys():
    exec('{} = payload["{}"]'.format(k, k))

In [3]:
word_embedding_dim = 300
embedding_maxfeatures = tokenizer.num_words
embeddings_path = 'data/glove.6B.300d.txt'
titles_maxlen = titles_train.shape[1]
lstm_latent_dim = 256

In [4]:
embedding_vectors = {}
weights_matrix = np.zeros((embedding_maxfeatures + 1, word_embedding_dim))

with open(embeddings_path, 'r') as f:
    for line in f:
        line_split = line.strip().split(" ")
        vec = np.array(line_split[1:], dtype=float)
        word = line_split[0]
        embedding_vectors[word] = vec

In [5]:
for word, i in tokenizer.word_index.items():
    embedding_vector = embedding_vectors.get(word)
    if embedding_vector is not None and i <= embedding_maxfeatures:
        weights_matrix[i] = embedding_vector

In [22]:
K.clear_session()
title_input = Input(shape=(titles_maxlen,), name='title_in')
title_embedding = Embedding(
    input_dim=tokenizer.num_words+1,
    output_dim=word_embedding_dim,
    weights=[weights_matrix],
    name='title_embedding',
    trainable=False
)(title_input)

# We don't care about the encoder sequence outputs
_, encoder_h, encoder_c = LSTM(
    lstm_latent_dim, return_state=True, name='encoder_lstm'
)(title_embedding)

encoder_states = [encoder_h, encoder_c]

# decoder_inputs will be just zeros
decoder_input = Input(shape=(titles_maxlen, 1), name='decoder_in')
decoder_output, _, _ = LSTM(
    lstm_latent_dim, return_sequences=True, return_state=True, name='decoder_lstm'
)(decoder_input, initial_state=encoder_states)

main_out = Dense(embedding_maxfeatures, activation='softmax', name='main_out')(decoder_output)

model = Model([title_input, decoder_input], main_out)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
title_in (InputLayer)           (None, 20)           0                                            
__________________________________________________________________________________________________
title_embedding (Embedding)     (None, 20, 300)      12000300    title_in[0][0]                   
__________________________________________________________________________________________________
decoder_in (InputLayer)         (None, 20, 1)        0                                            
__________________________________________________________________________________________________
encoder_lstm (LSTM)             [(None, 256), (None, 570368      title_embedding[0][0]            
__________________________________________________________________________________________________
decoder_ls

In [17]:
model_name = 'autoencoder_v0'
log_dir = "logs/autoencoder/scalars/{}_{}".format(model_name,
                                      datetime.datetime.now().strftime("%Y%m%d_%H%M%S"))
model_checkpoint_path = 'models/autoencoder/checkpoints/'+ model_name +'.{epoch:02d}-{val_loss:.2f}.hdf5'

callbacks = [
    TensorBoard(log_dir=log_dir),
    ModelCheckpoint(model_checkpoint_path)
]

In [18]:
def generate_batch(titles, batch_size):
    i = 0
    while True:
        x = titles[i:i+batch_size]
        decoder_inputs = np.zeros((batch_size, titles_maxlen, 1))
        y = np.zeros((batch_size, titles_maxlen, embedding_maxfeatures))
        for j in range(batch_size):
            for t in range(titles_maxlen):
                y[j, t, x[j][t]] = 1
        
        i = (i+batch_size) % len(titles)
        yield ([x, decoder_inputs], y)

In [10]:
batch_size = 32
epochs = 10

In [23]:
model.fit_generator(
    generate_batch(titles_train, batch_size),
    steps_per_epoch=len(titles_train) // batch_size,
    epochs=epochs,
    validation_data=generate_batch(titles_val, batch_size),
    validation_steps=len(titles_val) // batch_size,
    callbacks=callbacks
)

Epoch 1/10
 3349/34942 [=>............................] - ETA: 8:04:50 - loss: 4.7886 - acc: 0.3729

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



10951/34942 [========>.....................] - ETA: 6:07:56 - loss: 3.5436 - acc: 0.4985

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



13040/34942 [==========>...................] - ETA: 5:35:47 - loss: 3.3445 - acc: 0.5209

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



22738/34942 [==================>...........] - ETA: 3:06:19 - loss: 2.7422 - acc: 0.5897

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



31211/34942 [=========================>....] - ETA: 56:53 - loss: 2.4267 - acc: 0.6265

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [24]:
print('done')

done
